# Hyperpivot simplification

In the same way that we can use local complementation and pivoting to simplify ZX-diagrams and remove spiders with a Clifford phase from the diagram, we can also apply a hyperpivot rule on a hypergraph built using H-boxes. In this notebook we demonstrate this rewrite rules, and show that it suffices to reduce a class of benchmark circuits to a type of normal form.
For more information on how the hyperpivot works, see [this paper](https://arxiv.org/abs/2003.13564).

In [4]:
import sys, os; sys.path.insert(0, '..')
import pyzx as zx
import random
from fractions import Fraction
%config InlineBackend.figure_format = 'svg'

In [5]:
ccz = zx.qasm("""
qreg q[3];
ccz q[0],q[1],q[2];
""").to_graph(zh=True)
display(zx.draw(ccz))
print(ccz.to_matrix())

None

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j]]


In [6]:
g = zx.qasm("""
qreg q[3];

ccz q[0],q[1],q[2];
h q[2];
t q[2];
ccz q[0],q[1],q[2];
h q[2];
t q[1];
ccz q[0],q[1],q[2];
s q[2];
ccx q[0],q[1],q[2];
""").to_graph(zh=True)
zx.draw(g, labels=True)

In [7]:
h = g.copy()
zx.simplify.spider_simp(h)
zx.hsimplify.to_hypergraph_form(h)
zx.draw(h,labels=True)
print(zx.hsimplify.just_hpivot_simp(h))
zx.draw(h,labels=True)

True


The following strategy `hpivot_simp` combines hyperpivots with spider fusions in order to simplify the diagram as much as possible.

In [8]:
h = g.copy()
zx.hsimplify.hpivot_simp(h)
zx.draw(h, labels=True)

We see that in this case we have no internal spiders left: every spider is connected to a boundary. It turns out that this behaviour holds for many circuits as we will also see below.

In [9]:
zx.compare_tensors(g,h)

True

In [10]:
dir_fast_circuits = os.path.join('..', 'circuits', 'Fast')

In [12]:
d = os.path.join('..', 'circuits', 'Fast')
print('Circuit'.ljust(30) + '  qubits' + '   gates' + '    Z' + '       H' + '  reduced')
for f in os.listdir(d):
    f1 = os.path.join(d,f)
    if f.find('QFTAdd8') != -1: continue # takes too long
    if not os.path.isfile(f1) or f.find('before') == -1: continue
    print(f.ljust(30), end='')
    
    c = zx.Circuit.load(f1)
    for g in c.gates:
        if isinstance(g, zx.gates.ZPhase):
            g.phase = g.phase.limit_denominator(3628800)
        
    print(str(c.qubits).rjust(8), end='')
    print(str(len(c.gates)).rjust(8), end='')
    g = c.to_graph(zh=True)
    zx.hsimplify.hpivot_simp(g)
    g.normalize()
    z = len([v for v in g.vertices() if g.type(v) == 1])
    h = len([v for v in g.vertices() if g.type(v) == 3])
    print(str(z).rjust(5), end='')
    print(str(h).rjust(8), end='  ')
    
    print(g.qubit_count() * 2 == z)

Circuit                         qubits   gates    Z       H  reduced
Adder8_before                       23     105   46      44  True
adder_8_before                      24     216   48     101  True
barenco_tof_10_before               19      66   38       9  True
barenco_tof_3_before                 5      10   10       3  True
barenco_tof_4_before                 7      18   14       1  True
barenco_tof_5_before                 9      26   18       1  True
csla_mux_3_original_before          15      50   30      35  True
csum_mux_9_corrected_before         30      56   60      28  True
gf2^10_mult_before                  30     309   60     127  True
gf2^4_mult_before                   12      51   24      20  True
gf2^5_mult_before                   15      79   30      31  True
gf2^6_mult_before                   18     113   36      47  True
gf2^7_mult_before                   21     153   42      67  True
gf2^8_mult_before                   24     213   48     113  True
gf2^9_m